# Chebyshev center

Find the center and radius of the largest sphere that fits inside the polyhedron defined by the inequalities
$$ \left\{ 2 x_1 - x_2 + 2 x_3 \le 2,\quad
-x_1 + 2 x_2 + 4 x_3 \le 16,\quad
x_1 + 2 x_2 - 2 x_3 \le 8,\quad
x_1 \ge 0,\,
x_2 \ge 0,\,
x_3 \ge 0 \right\}$$

## Optimization problem formulation

The Chebyshev center problem can be formulated as the following linear program:

$\max_{x,r} ~~r $

subject to

$ a_jx + ||a_j||r\leq b_j\quad$ for $j=1,...,6$ 

where the vectors $a$ are the normal vector for each hyperplane and $b$ is the constant on the right hand side.

$a_1 = [ 2~~-\!1~~2]$,

$a_2 = [ -\!1~~2~~4]$, 

$a_3 = [ 1~~2~~-\!2]$, 

$a_4 = [ -\!1~~0~~0]$, 

$a_5 = [ 0~~-\!1~~0]$, 

$a_6 = [ 0~~0~~-\!1]$, and

$b = [2~~16~~8~~0~~0~~0]$.

## JuMP implementation

In [1]:
# If you haven't yet installed the linear algebra package, do the standard thing:
using Pkg
Pkg.add("LinearAlgebra")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


    Updating `~/.julia/environments/v1.10/Project.toml`
  [37e2e46d] + LinearAlgebra
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
# Matrices A, b defining the inequalities (each row vector is a normal vector to a hyperplane!)
A = [2 -1 2; -1 2 4; 1 2 -2; -1 0 0; 0 -1 0; 0 0 -1];
b = [2; 16; 8; 0; 0; 0]

using JuMP, HiGHS, LinearAlgebra

m = Model(HiGHS.Optimizer)
@variable(m, r >= 0)           # radius of the sphere
@variable(m, x[1:3]>=0)           # coordinates of center
for i = 1:size(A,1) # number of rows
    @constraint(m, A[i,:]'*x + r*norm(A[i,:]) <= b[i]) # Note fancy matlab style notation
end
@objective(m, Max, r)     # maximize radius of the sphere

optimize!(m)
center = value.(x)
radius = value(r)

println("HiGHS LP solver terminated with status ", termination_status(m))
println("The coordinates of the Chebyshev center are: ", center)
println("The largest possible radius is: ", radius)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
6 rows, 4 cols, 18 nonzeros
6 rows, 4 cols, 18 nonzeros
Presolve : Reductions: rows 6(-0); columns 4(-0); elements 18(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -9.9999824725e-01 Ph1: 6(13.5826); Du: 1(0.999998) 0s
          6     7.5000000000e-01 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 6
Objective value     :  7.5000000000e-01
HiGHS run time      :          0.00
HiGHS LP solver terminated with status OPTIMAL
The coordinates of the Chebyshev center are: 

[0.7499999999999999, 3.2499999999999996, 0.75]
The largest possible radius is: 0.7500000000000002
